In [1]:
import pandas as pd
import numpy as np
import camelot.io as camelot
import sys

 # adding folders up to system path to access functions
sys.path.insert(0, '../../../')
from driveFolderHandler import downloadTempDriveFolder, deleteTempDriveFolder

In [2]:
path = "../temp-docs"
downloadTempDriveFolder(folderID='1eS9Tan2OI_y0K_ZWr5koyCVHzFggFlKn',
                        path = path)

Retrieving folder contents


Processing file 18iEKHNFKevLw_HXeyd5QTd7ZHXB6HQIB 62-552.200.doc
Processing file 1tvcfSGYHSjcLgFvZikKc04buDtaNeEV1 62-552.300.doc
Processing file 12K_FHWh24gdqB3ufiaxX2VHJP4s1RxxV DWSRF IUP SFY 2023 FINAL_0.pdf
Processing file 1Hl1Nooml5DyimowbkRBVGUek5Nu-Mnty DWSRF SFY2023 Priority List.pdf
Processing file 1B1U1_qvzrlJMKCG9Y0kxnPkEgtn6EMF8 FINAL EC IUP DWSRF 2022 11072023.pdf
Processing file 1_AG1DqdsND1dgKVJvaIwWWvwxfW3FlhN Final LSL IUP DWSRF FFY2022-2023 rev 7-12-2023.pdf
Processing file 1w8_3N8wAyKf6byloXp_7FFnsoZHz3R3w FL_IUPandPPL_2023.pdf
Building directory structure completed


Retrieving folder contents completed
Building directory structure
Downloading...
From: https://drive.google.com/uc?id=18iEKHNFKevLw_HXeyd5QTd7ZHXB6HQIB
To: /Users/emmalitsai/dw-dashboard/year1/FL/temp-docs/62-552.200.doc
100%|██████████| 32.8k/32.8k [00:00<00:00, 6.25MB/s]
Downloading...
From: https://drive.google.com/uc?id=1tvcfSGYHSjcLgFvZikKc04buDtaNeEV1
To: /Users/emmalitsai/dw-dashboard/year1/FL/temp-docs/62-552.300.doc
100%|██████████| 67.1k/67.1k [00:00<00:00, 1.24MB/s]
Downloading...
From: https://drive.google.com/uc?id=12K_FHWh24gdqB3ufiaxX2VHJP4s1RxxV
To: /Users/emmalitsai/dw-dashboard/year1/FL/temp-docs/DWSRF IUP SFY 2023 FINAL_0.pdf
100%|██████████| 832k/832k [00:00<00:00, 3.88MB/s]
Downloading...
From: https://drive.google.com/uc?id=1Hl1Nooml5DyimowbkRBVGUek5Nu-Mnty
To: /Users/emmalitsai/dw-dashboard/year1/FL/temp-docs/DWSRF SFY2023 Priority List.pdf
100%|██████████| 73.6k/73.6k [00:00<00:00, 993kB/s]
Downloading...
From: https://drive.google.com/uc?id=1B1U1_qvzrlJMKCG9Y0k

Files stored in ../temp-docs


Download completed


# Florida

### Florida Table 1
The first table from Florida, which covers three pages, seems to be scanned in, so it can be particularly difficult to make columns and rows exactly match.
Given that the PPL only covers 3 pages, each is individually corrected so that edits can be reproducible and edited based on review of the original document.

In [3]:
fl_pdfs = camelot.read_pdf(path+"/FL_IUPandPPL_2023.pdf",
                                pages='27-30', flavor = 'stream', row_tol=25)
print(len(fl_pdfs))

4


In [4]:
# take the 0th page of the first table
fl1_0 = fl_pdfs[0].df

# drop column name row
fl1_0 = fl1_0.iloc[1:,].copy().reset_index(drop=True)

fl1_0.columns = ['Tier', 'Priority.Score', 'Applicant.ProjectNbr', 'Project.Type', 'Project.Description', 
                    'Adoption.Date', 'Application.Deadline', 'Agreement.Deadline', 'Authorized.Loan.Amount',
                    'Principal.Forgiveness.Amt', 'Amount.To.Be.Repaid']

fl1_0 = fl1_0.replace("\n", "", regex=True)

#manually combine rows where content spills over
fl1_0.iloc[3,2] = fl1_0.iloc[2,2] + " " + fl1_0.iloc[3,2]
fl1_0.iloc[3,4] = fl1_0.iloc[2,4] + " " + fl1_0.iloc[3,4]
fl1_0.iloc[3,4] = fl1_0.iloc[3,4] + " " + fl1_0.iloc[4,4]
fl1_0.iloc[11,4] = fl1_0.iloc[11,4] + " " + fl1_0.iloc[12,4]

#manually remove data from where total row merged into last project row
fl1_0.iloc[17,1] = fl1_0.iloc[17,1].replace(" TOTAL AWARDED SEGMENTS:", "")
fl1_0.iloc[17,8] = fl1_0.iloc[17,8].replace(" $29,634,701", "")
fl1_0.iloc[17,9] = fl1_0.iloc[17,9].replace(" $0", "")
fl1_0.iloc[17,10] = fl1_0.iloc[17,10].replace(" $29,634,701", "")

fl1_0.drop([2,4,12], inplace=True)

fl1_0['Tier'] = fl1_0['Tier'].str.replace("Tier", "Tier ")

# separate applicant name and project number then drop combined column
fl1_0[['Applicant', 'Project.Number']] = fl1_0['Applicant.ProjectNbr'].str.extract(r'(.*)\s(\d+)', expand=True)
fl1_0.drop(columns=['Applicant.ProjectNbr'], inplace=True)

#reorder columns for concat
fl1_0 = fl1_0[['Tier', 'Priority.Score', 'Applicant', 'Project.Number', 'Project.Type', 'Project.Description', 'Adoption.Date', 
                      'Application.Deadline', 'Agreement.Deadline', 'Authorized.Loan.Amount',
                      'Principal.Forgiveness.Amt', 'Amount.To.Be.Repaid']]

# manually fix combined columns and rows: et added:v 
fl1_0.iloc[2,5] = "Source, Treatment, Distribution (SE Wellfield, RO and Transmission)"
fl1_0.iloc[3,5] = "Treatment and  Disposal (RO WTP and  DIW Mod and  reject)"
fl1_0.iloc[9,5] = "Treatment (DIW Pump Station & RO Effluent Removal)"
fl1_0.iloc[10,5] = "Distribution (Extend water to Lighthouse subd)"
fl1_0.iloc[12,5] = "Treatment (GAC Filters)"
fl1_0.iloc[13,5] = "Treatment (WTP Improvements Phase 1A)"
# et added: ^

fl1_0.head()

,Tier,Priority.Score,Applicant,Project.Number,Project.Type,Project.Description,Adoption.Date,Application.Deadline,Agreement.Deadline,Authorized.Loan.Amount,Principal.Forgiveness.Amt,Amount.To.Be.Repaid
0,Tier 1,656,"Moore Creek- Mt. Carmel Utilities, Inc. *",57100,DW/Con,Source (Construction of 2 Wells),08/11/2021 02/22/2022,12/09/2021,AWARDED,"$62,839",$0,"$62,839"
1,Tier 1,561,Haines City,53045,DW/Con,Treatment (WTPs Upgrades),11/10/2021,03/10/2022,06/08/2022,"$1,962,591",$0,"$1,962,591"
3,Tier 1,421,Polk Regional Water Cooperative,53200,DW/Plan/Des,"Source, Treatment, Distribution (SE Wellfield,...",02/23/2022,06/23/2022,09/21/2022,"$20,000,000",$0,"$20,000,000"
5,Tier 1,414,Stuart,43045,DW/Con,Treatment and Disposal (RO WTP and DIW Mod a...,02/23/2022,06/23/2022,09/21/2022,"$4,888,901",$0,"$4,888,901"
6,Tier 1,401,Hollywood,06049,DW/Con,Treatment (Water Main Replacement),08/11/2021,12/09/2021,AWARDED 10/25/2021,"$668,257",$0,"$668,257"


In [5]:
fl1_1 = fl_pdfs[1].df

# drop column name row
fl1_1 = fl1_1.iloc[1:,].copy().reset_index(drop=True)

#fl1_1.columns = ['Tier', 'Priority.Score', 'Applicant.ProjectNbr', 'Project.Type', 'Project.Description', 
#                    'Adoption.Date', 'Application.Deadline', 'Agreement.Deadline', 'Authorized.Loan.Amount',
#                    'Principal.Forgiveness.Amt', 'Amount.To.Be.Repaid']

fl1_1 = fl1_1.replace("\n", "", regex=True)

# manually fix combined columns and rows
fl1_1.iloc[0,1] = "564"

fl1_1.iloc[3,2] = "American Beach Water and Sewer District* 45051"
fl1_1.iloc[7,2] = "Eagle Lake* 53091"
fl1_1.iloc[17,2] = "Big Bend Water Authority* 15052"
fl1_1.iloc[18,2] = "Bowling Green* 25028"
fl1_1.iloc[21,2] = "Holt Water Works, Inc.* 46023"

fl1_1.iloc[0,3] = "DW/Plan/Des"

fl1_1.iloc[0,4] = "Distribution (Miama Gardens)"
fl1_1.iloc[1,4] = "Distribution (Water Line and Meter Replacement)"
fl1_1.iloc[3,4] = "Distribution (Connecting to	Existing Water System)"
# et added:v
fl1_1.iloc[5,4] = "Distribution (Miami Gardens)"
fl1_1.iloc[6,4] = "Distribution (Water Distribution System Improvements)"
# et added:^ 
fl1_1.iloc[7,4] = "Treatment (Green Acres WTP Improvements)"
fl1_1.iloc[9,4] = "Distribution (Water Distribution System Improvements)"
fl1_1.iloc[10,4] = "Treatment (WTP#3 Upgrades Ph1)"
# et added:v
fl1_1.iloc[11,4] = "Distribution (NW Water Main Extension)"
fl1_1.iloc[11,5] = "02/23/2022"
# et added:^ 
fl1_1.iloc[14,4] = "Treatment & Distribution (Pipe Rehabilitation & Looping)"
fl1_1.iloc[15,4] = "Treatment (RO WTP, WRF DIW Mod, RO Concentrate	Pipe)"
fl1_1.iloc[17,4] = "Distribution (Water Distribution Replacement & Meters)"
fl1_1.iloc[18,4] = "Distribution (Water main improvements Ph1)"
fl1_1.iloc[20,4] = "Distribution (Water Distribution Replacement)"
fl1_1.iloc[21,4] = "Source, Treatment, Storage (Well, WTP and EST)"

#drop spillover rows
fl1_1.drop([2,8,13, 16, 19, 22], inplace=True)

fl1_1[0] = fl1_1[0].str.replace("Tier", "Tier ")

fl1_1.reset_index(drop=True)

#rename columns for transformations
fl1_1.columns = ['Tier', 'Priority.Score', 'Applicant.ProjectNbr', 'Project.Type', 'Project.Description', 'Adoption.Date', 
                      'Application.Deadline', 'Agreement.Deadline', 'Authorized.Loan.Amount',
                      'Principal.Forgiveness.Amt', 'Amount.To.Be.Repaid']

# separate applicant name and project number then drop combined column
fl1_1[['Applicant', 'Project.Number']] = fl1_1['Applicant.ProjectNbr'].str.extract(r'(.*)\s(\d+)', expand=True)
fl1_1.drop(columns=['Applicant.ProjectNbr'], inplace=True)

#reorder columns for concat
fl1_1 = fl1_1[['Tier', 'Priority.Score', 'Applicant', 'Project.Number', 'Project.Type', 'Project.Description', 'Adoption.Date', 
                      'Application.Deadline', 'Agreement.Deadline', 'Authorized.Loan.Amount',
                      'Principal.Forgiveness.Amt', 'Amount.To.Be.Repaid']]

fl1_1.head()

,Tier,Priority.Score,Applicant,Project.Number,Project.Type,Project.Description,Adoption.Date,Application.Deadline,Agreement.Deadline,Authorized.Loan.Amount,Principal.Forgiveness.Amt,Amount.To.Be.Repaid
0,Tier 2,564,Opa-Locka*,13038,DW/Plan/Des,Distribution (Miama Gardens),02/23/2022,06/23/2022,09/21/2022,"$152,600","$76,300","$76,300"
1,Tier 2,511,"Mobile Manor, Inc*",36087,DW/Con,Distribution (Water Line and Meter Replacement),08/11/2021 02/03/2022,12/09/2021,AWARDED,"$2,315,728","$2,084,155","$231,573"
3,Tier 3,483,American Beach Water and Sewer District*,45051,DW/Con,Distribution (Connecting to\tExisting Water Sy...,08/11/2021,12/09/2021,03/09/2022,"$2,523,500","$2,064,825","$458,675"
4,Tier 3,480,Fellsmere*,31031,DW/Con,Distribution (AMR Meter Replacement),02/23/2022,06/23/2022,09/21/2022,"$605,981","$545,383","$60,598"
5,Tier 3,464,Opa-Locka*,13038,DW/Con,Distribution (Miami Gardens),02/23/2022,06/23/2022,09/21/2022,"$1,581,996","$1,423,796","$158,200"


In [6]:
fl1_2 = fl_pdfs[2].df

# drop column name row
fl1_2 = fl1_2.iloc[1:15,].copy().reset_index(drop=True)

# replace linebreaks
fl1_2 = fl1_2.replace("\n", "", regex=True)

# manually move system names out of 0 column for two instances
fl1_2.iloc[0,0] = "Tier3 353"
fl1_2.iloc[0,1] = "Mulberry* 53122"

fl1_2.iloc[1,0] = "Tier3 338"
fl1_2.iloc[1,1] = "Springfield* 03051"

# manually set Tier column which erroneously merged
fl1_2['Tier'] = "Tier 3"

# remove Tier from Priority.Score
fl1_2[0] = fl1_2[0].str.replace("Tier3 ", "")

#manually combine rows erroneously split, then drop excess columns
fl1_2.iloc[9,1] = fl1_2.iloc[9,1] + " " + fl1_2.iloc[10,1]
fl1_2.iloc[11,3] = fl1_2.iloc[10,3] + " " + fl1_2.iloc[11,3]
fl1_2.iloc[12,3] = fl1_2.iloc[12,3] + " " + fl1_2.iloc[13,3]

fl1_2.drop([10,13], inplace=True)

#rename columns for transformations
fl1_2.columns = ['Priority.Score', 'Applicant.ProjectNbr', 'Project.Type', 'Project.Description', 'Adoption.Date', 
                      'Application.Deadline', 'Agreement.Deadline', 'Authorized.Loan.Amount',
                      'Principal.Forgiveness.Amt', 'Amount.To.Be.Repaid', 'Tier']

# separate applicant name and project number then drop combined column
fl1_2[['Applicant', 'Project.Number']] = fl1_2['Applicant.ProjectNbr'].str.extract(r'(.*)\s(\d+)', expand=True)
fl1_2.drop(columns=['Applicant.ProjectNbr'], inplace=True)

#reorder columns for concat
fl1_2 = fl1_2[['Tier', 'Priority.Score', 'Applicant', 'Project.Number', 'Project.Type', 'Project.Description', 'Adoption.Date', 
                      'Application.Deadline', 'Agreement.Deadline', 'Authorized.Loan.Amount',
                      'Principal.Forgiveness.Amt', 'Amount.To.Be.Repaid']]

# manually fix combined columns and rows: et added:v 
fl1_2.iloc[0,5] = "Distribution (Water main  improvements)"
# et added: ^

fl1_2.head()

,Tier,Priority.Score,Applicant,Project.Number,Project.Type,Project.Description,Adoption.Date,Application.Deadline,Agreement.Deadline,Authorized.Loan.Amount,Principal.Forgiveness.Amt,Amount.To.Be.Repaid
0,Tier 3,353,Mulberry*,53122,DW/Plan,Distribution (Water main improvements),02/23/2022,06/23/2022,09/21/2022,"$67,000","$33,500","$33,500"
1,Tier 3,338,Springfield*,03051,DW/Con,Treatment (Water System Rehabilitation),08/11/2021,12/09/2021,03/09/2022,"$2,038,000","$1,236,633","$801,367"
2,Tier 3,337,Gulf County,23022,DW/Con,Treatment (Booster Station),02/23/2022,06/23/2022,09/21/2022,"$932,200","$393,625","$538,575"
3,Tier 3,320,Dania Beach,06123,DW/Con,Source (Capacity Purchase in C51 Reservoir),08/11/2021,12/09/2021,03/09/2022,"$4,600,000",$0,"$4,600,000"
4,Tier 3,320,Newberry,01081,DW/Plan,Storage (Water tank replacement),02/23/2022,06/23/2022,09/21/2022,"$150,000",$0,"$150,000"


In [7]:
# concat all fl_1 dfs for final output
fl_1 = pd.concat([fl1_0, fl1_1, fl1_2]).reset_index(drop=True)
fl_1.sample(10)

,Tier,Priority.Score,Applicant,Project.Number,Project.Type,Project.Description,Adoption.Date,Application.Deadline,Agreement.Deadline,Authorized.Loan.Amount,Principal.Forgiveness.Amt,Amount.To.Be.Repaid
2,Tier 1,421,Polk Regional Water Cooperative,53200,DW/Plan/Des,"Source, Treatment, Distribution (SE Wellfield,...",02/23/2022,06/23/2022,09/21/2022,"$20,000,000",$0,"$20,000,000"
39,Tier 3,156,Jackson County*,32038,DW/Con,Distribution (Water main to Indian Springs),02/23/2022,06/23/2022,09/21/2022,"$6,696,354","$2,250,000","$4,446,354"
12,Tier 1,152,Palatka,54025,DW/Con,Treatment (GAC Filters),05/25/2022,09/22/2022,12/21/2022,"$1,565,700",$0,"$1,565,700"
43,Tier 3,100,Venice,58048,DW/Con,Treatment (WTP Switchgear Replacement ),08/11/2021,12/09/2021,AWARDED 01/14/2022,"$17,461,565",$0,"$17,461,565"
35,Tier 3,320,Dania Beach,06123,DW/Con,Source (Capacity Purchase in C51 Reservoir),08/11/2021,12/09/2021,03/09/2022,"$4,600,000",$0,"$4,600,000"
26,Tier 3,421,Orange City*,64204,DW/Con,Treatment & Distribution (Pipe Rehabilitation ...,08/11/2021,12/09/2021,AWARDED 01/28/2022,"$17,250,898","$2,250,000","$15,000,898"
36,Tier 3,320,Newberry,01081,DW/Plan,Storage (Water tank replacement),02/23/2022,06/23/2022,09/21/2022,"$150,000",$0,"$150,000"
16,Tier 2,511,"Mobile Manor, Inc*",36087,DW/Con,Distribution (Water Line and Meter Replacement),08/11/2021 02/03/2022,12/09/2021,AWARDED,"$2,315,728","$2,084,155","$231,573"
23,Tier 3,436,Belleview*,42077,DW/Con,Treatment (WTP#3 Upgrades Ph1),02/23/2022,06/23/2022,09/21/2022,"$6,072,651","$2,250,000","$3,822,651"
6,Tier 1,383,Sanford,59019,DW/Con,Treatment (Main WTP Improvements),08/11/2021,12/09/2021,AWARDED 01/26/2022,"$6,457,000",$0,"$6,457,000"


In [8]:
# et edit to new file location: 
fl_1.to_csv("../data/9-Florida_ppl_fundable.csv")

### Florida Table 2

In [9]:
fl_pdfs_2 = camelot.read_pdf("https://floridadep.gov/sites/default/files/DWSRF%20IUP%20SFY%202023%20FINAL_0.pdf",
                                pages='31', flavor = 'stream')
print(len(fl_pdfs_2))

1


In [10]:
fl2_0 = fl_pdfs_2[0].df

# remove header and footer
fl2_0 = fl2_0.iloc[3:34,].copy().reset_index(drop=True)

#remove the $ suffix from Awarded Amount column where it is included
fl2_0[3] = fl2_0[3].str.replace(" \$", "")

# add $ column to prefix of Awarded Amount column
fl2_0[3] = fl2_0[2] + fl2_0[3]

# drop $ columns
fl2_0.drop(columns=[2,4], inplace=True)

fl2_0.columns = ['Sponsor.Name', 'Funding.#', 'Awarded.Amount', 'PF.Awarded', 'Type']

# et edit to new file location: 
fl2_0.to_csv("../data/9-Florida_PPL_EquivalencyProjects.csv", index=False)

In [7]:
deleteTempDriveFolder(path)

Deleted folder and all files within ../temp-docs
